# Manejo y Optimización de Particiones en Spark

En este notebook aprenderás a:
- Visualizar y modificar el número de particiones de un DataFrame.
- Usar las funciones `repartition` y `coalesce`.
- Guardar datos con particionado por columna y medir el impacto en el rendimiento.
Trabajaremos con el dataset MovieLens.


In [ ]:
# Cargar datos MovieLens (ratings.csv) suponiendo que está en /dbfs/FileStore/movielens/ratings.csv
df = spark.read.csv("/dbfs/FileStore/movielens/ratings.csv", header=True, inferSchema=True)
df.printSchema()
df.count()

# Ver número de particiones
print("Particiones iniciales:", df.rdd.getNumPartitions())

# Cambiar el número de particiones a 8
df_repart = df.repartition(8)
print("Después de repartition(8):", df_repart.rdd.getNumPartitions())

# Reducir a 2 particiones con coalesce
df_coalesce = df_repart.coalesce(2)
print("Después de coalesce(2):", df_coalesce.rdd.getNumPartitions())

# Guardar datos particionando por 'userId'
output_path = "/dbfs/FileStore/movielens/ratings_by_user"
df.write.mode("overwrite").partitionBy("userId").parquet(output_path)

# Explorar estructura de carpetas (solo funciona en Databricks)
display(dbutils.fs.ls("/FileStore/movielens/ratings_by_user"))

# BONUS: Medición de tiempo de escritura con diferentes particiones
import time
start = time.time()
df_repart.write.mode("overwrite").parquet("/dbfs/FileStore/movielens/test_repart")
print("Tiempo con repartition(8):", time.time() - start, "segundos")
